In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
from fastai.custom_extensions import *

In [ ]:
# !pip install git+https://github.com/ageitgey/face_recognition_models

In [5]:
from face_recognition import *

ModuleNotFoundError: No module named 'face_recognition'

In [9]:
PATH = "data/cricket_baseball/baseball/"

In [26]:
image = face_recognition.load_image_file(f'{PATH}julio-urias-dodgers_0.jpg')
face_locations = face_recognition.face_locations(image)

In [27]:
for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = PIL.Image.fromarray(face_image)
    pil_image.show()

In [29]:
face_detector = dlib.get_frontal_face_detector()

In [30]:
detected_faces = face_detector(image, 1)

In [36]:
cnn_face_detection_model = face_recognition.face_recognition_models.cnn_face_detector_model_location

In [37]:
cnn_face_detector = dlib.cnn_face_detection_model_v1(cnn_face_detection_model)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. dlib.cnn_face_detection_model_v1(arg0: str)

Invoked with: <function cnn_face_detector_model_location at 0x7f94254afbf8>

### Data preparation

In [ ]:
train = pd.read_csv(f'{PATH}train.csv')
test = pd.read_csv(f'{PATH}test.csv')

In [ ]:
# create labels df
# label_df = train['label']
label_df = train.iloc[:,:1]

# remove labels from training data
del train['label']
# img_df = train.iloc[:,1:]     # train.drop(labels = ["label"],axis = 1)

In [ ]:
# labels = pd.DataFrame(label_df)
label_df.insert(0,'fnames', range(0, len(label_df)))
# label_df.to_csv(f'{PATH}labels.csv', index=False)

In [ ]:
# view images w/ labels
i = 0
img = train.iloc[i].values.reshape(28,28)
plt.imshow(img)
plt.title(label_df[i])

In [ ]:
# check count distributions
# normal distribution, looks good
label_df.hist()

In [ ]:
# Check for null or missing values
label_df.isnull().any().describe()
train.isnull().any().describe()

In [ ]:
# Normalize the data -> currently in grayscale.  we only need black/white (binary)
# train = train.applymap(lambda x: 1 if x>0 else 0)
# test = test.applymap(lambda x: 1 if x>0 else 0)

In [ ]:
# Reshape 28 x 28 arrays
train_imgs = train.values.reshape(-1, 28, 28)
test_imgs = test.values.reshape(-1, 28, 28)

In [ ]:
label_df.loc[3]['label']

In [ ]:
# Create jpgs in train/
os.makedirs(f'{PATH}train/', exist_ok=True)

for i in range(0,len(train_imgs)):
    img = train_imgs[i][:]
    # img = img.reshape(28,28)
    img = np.stack((img,)*3,axis = -1)
    img = np.uint8(img)
    Image.fromarray(img).save(f'{PATH}train/' + str(i) + '.jpg', 'JPEG')

In [ ]:
# Create jpgs in test/
os.makedirs(f'{PATH}test/', exist_ok=True)

for i in range(0,len(test_imgs)):
    img = test_imgs[i][:]
#     img = img.reshape(28,28)
    img = np.stack((img,)*3,axis = -1)
    img = np.uint8(img)
    Image.fromarray(img).save(f'{PATH}test/' + str(i) + '.jpg', 'JPEG')

In [ ]:
i = 33
img = plt.imread(f'{PATH}train/{i}.jpg')
plt.imshow(img)
plt.title(label_df.loc[i]['label'])

## Training

In [ ]:
bs = 256
sz = 28
arch = resnet34

In [ ]:
label_path = f'{PATH}labels.csv'
n = len(list(open(label_path)))-1
val_idxs = get_cv_idxs(n)

In [ ]:
def get_data(sz,bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', label_path, bs=bs, tfms=tfms, val_idxs=val_idxs,
                                        suffix='.jpg', test_name='test')
    return data

In [ ]:
data = get_data(sz,bs)

In [ ]:
learn = ConvLearner.pretrained(arch, data)

In [ ]:
learn.lr_find()

In [ ]:
def plot_loss_change(sched, sma=1, n_skip=20, y_lim=(-0.01,0.01)):
    """
    Plots rate of change of the loss function.
    Parameters:
        sched - learning rate scheduler, an instance of LR_Finder class.
        sma - number of batches for simple moving average to smooth out the curve.
        n_skip - number of batches to skip on the left.
        y_lim - limits for the y axis.
    """
    derivatives = [0] * (sma + 1)
    for i in range(1 + sma, len(learn.sched.lrs)):
        derivative = (learn.sched.losses[i] - learn.sched.losses[i - sma]) / sma
        derivatives.append(derivative)
        
    plt.ylabel("d/loss")
    plt.xlabel("learning rate (log scale)")
    plt.plot(learn.sched.lrs[n_skip:], derivatives[n_skip:])
    plt.xscale('log')
    plt.ylim(y_lim)

In [ ]:
learn.sched.plot()

In [ ]:
plot_loss_change(learn.sched, sma=20, y_lim=(-0.03,0.01))

In [ ]:
lr = 0.06

In [ ]:
learn.fit(lr, 3, cycle_len=1)

In [ ]:
learn.fit(lr, 5, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save('resnet34_v1_48')

In [ ]:
learn.load('resnet34_v1_48')

In [ ]:
log_preds,y = learn.TTA(is_test=True)

In [ ]:
probs = np.mean(np.exp(log_preds),0)

In [ ]:
choices = np.argmax(probs, axis=1)
# Returns the indices of the maximum values along an axis

In [ ]:
choices

In [ ]:
data.test_ds.fnames

In [ ]:
df = pd.DataFrame(choices)
df.insert(0, 'ImageId', [(int(o[5:-4])+1) for o in data.test_ds.fnames])
df.columns = ["ImageId", "Label"]

In [ ]:
df = df.sort_index(by=['ImageId'])

In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}subm/'
os.makedirs(SUBM, exist_ok=True)
filename = 'subm_v1.csv'
df.to_csv(f'{SUBM}{filename}', index=False)

In [ ]:
# submit to kaggle via cli
! kg submit {SUBM+filename} -c digit-recognizer